![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Paso 1: Instalación de dependencias y configuración del entorno en GPU

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 33kB/s 
     |████████████████████████████████| 419kB 41.1MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
     |████████████████████████████████| 61kB 28.4MB/s 


In [0]:
!pip install tqdm

### Descargar el dataset de Perros vs Gatos

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-08-01 13:55:17--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.79.128, 2a00:1450:4013:c00::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.79.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  34.4MB/s    in 1.9s    

2019-08-01 13:55:19 (34.4 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Paso 2: Pre procesado del data set

### Importar las dependencias del proyecto

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'1.14.0'

### Descomprimir el dataset de Perros vs Gatos

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Configurar las rutas al dataset

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Construir el Modelo

### Cargar un modelo pre entrenado (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

W0801 14:13:28.105518 139643395622784 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


9412608/9406464 [==============================] - 1s 0us/step


In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Congelar el modelo base

In [0]:
base_model.trainable = False

### Definir la cabecera personalizada para nuestra red neuronal

In [0]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(?, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Definir el modelo

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compilar el modelo

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

W0801 14:38:35.068260 139643395622784 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Crear generadores de datos

Redimensionar imágenes

    Las grandes arquitecturas pre-entrenadas solamente soportan cierto tipo de tamaños de imágenes.
    
Por ejemplo: MobileNet (la arquitectura que nosotros usamos) soporta: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [0]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Entrenar el modelo 

In [0]:
model.fit_generator(train_generator, epochs=8, validation_data=valid_generator)

Epoch 1/8
16/16 [==============================] - 9s 560ms/step - loss: 0.4902 - acc: 0.7640 - val_loss: 0.4279 - val_acc: 0.8090
Epoch 2/8
16/16 [==============================] - 7s 412ms/step - loss: 0.4526 - acc: 0.7955 - val_loss: 0.4030 - val_acc: 0.8210
Epoch 3/8
16/16 [==============================] - 6s 405ms/step - loss: 0.4297 - acc: 0.8100 - val_loss: 0.3745 - val_acc: 0.8320
Epoch 4/8
16/16 [==============================] - 7s 412ms/step - loss: 0.4036 - acc: 0.8270 - val_loss: 0.3584 - val_acc: 0.8410
Epoch 5/8
16/16 [==============================] - 7s 415ms/step - loss: 0.3779 - acc: 0.8455 - val_loss: 0.3405 - val_acc: 0.8530
Epoch 6/8
16/16 [==============================] - 7s 419ms/step - loss: 0.3560 - acc: 0.8590 - val_loss: 0.3235 - val_acc: 0.8650
Epoch 7/8
16/16 [==============================] - 7s 407ms/step - loss: 0.3439 - acc: 0.8690 - val_loss: 0.3107 - val_acc: 0.8710
Epoch 8/8
16/16 [==============================] - 7s 407ms/step - loss: 0.3278 - a

### Evaluar el modelo de aprendizaje por transferencia

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8809999823570251


## Puesta a punto de parámetros


Un par de cosas:

- NUNCA HAY QUE USAR la puesta a punto (fine tuning) de parámetros en toda la red neuronal: con algunas de las capas superiores (las finales) es más que suficiente suficiente. En la mayoría de casos, son las más especializadas. El objetivo del fine tuning es adaptar esa parte específica de la red neuronal para nuestro nuevo dataset específico.
- Empezar con la puesta a punto DESPUÉS de haber finalizado la fase de aprendizaje por transferencia. Si intentamos hacer el Fine tuning inmediatamente, los gradientes serán muy diferentes entre nuestra cabecera personalizada de la red neuronal y las nuevas capas no congeladas del modelo base.

### Descongelar unas cuantas capas superiores del modelo

In [0]:
base_model.trainable = True

In [0]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [0]:
fine_tune_at = 100

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compilar el modelo para la puesta a punto

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Puesta a punto

In [0]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 26s 2s/step - loss: 4.4638e-04 - accuracy: 1.0000 - val_loss: 0.4165 - val_accuracy: 0.9390
Epoch 2/5
16/16 [==============================] - 7s 454ms/step - loss: 1.3087e-04 - accuracy: 1.0000 - val_loss: 0.5169 - val_accuracy: 0.9290
Epoch 3/5
16/16 [==============================] - 7s 428ms/step - loss: 6.1403e-05 - accuracy: 1.0000 - val_loss: 0.4853 - val_accuracy: 0.9370
Epoch 4/5
16/16 [==============================] - 7s 449ms/step - loss: 2.1485e-05 - accuracy: 1.0000 - val_loss: 0.5115 - val_accuracy: 0.9340
Epoch 5/5
16/16 [==============================] - 7s 440ms/step - loss: 8.7441e-06 - accuracy: 1.0000 - val_loss: 0.5752 - val_accuracy: 0.9340


### Evaluar el modelo re calibrado

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9340000152587891
